In [34]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import joblib
price_scaler = joblib.load("../../data/close_price_scaler.pkl")



# =========================
# Paths
# =========================
MODEL_PATH = "currentOlsSolution.pkl"
ACTIVATION_PATH = "../../data/activation_data.csv"
TRAIN_PATH = "../../data/training_data.csv"

# =========================
# Load model & data
# =========================
ols_model = sm.load(MODEL_PATH)

activation_df = pd.read_csv(ACTIVATION_PATH)
train_df = pd.read_csv(TRAIN_PATH)

In [35]:
TARGET = "Close_Price"
DROP_COLS = ["Date"]

# =========================
# Rebuild TRAINING features (reference)
# =========================
train_df["Date"] = pd.to_datetime(train_df["Date"])
train_df["Year"]  = train_df["Date"].dt.year
train_df["Month"] = train_df["Date"].dt.month
train_df["Day"]   = train_df["Date"].dt.day

X_train_ref = train_df.drop(columns=DROP_COLS + [TARGET])

# =========================
# Rebuild ACTIVATION features
# =========================
activation_df["Date"] = pd.to_datetime(activation_df["Date"])
activation_df["Year"]  = activation_df["Date"].dt.year
activation_df["Month"] = activation_df["Date"].dt.month
activation_df["Day"]   = activation_df["Date"].dt.day

X_activation = activation_df.drop(columns=DROP_COLS + [TARGET], errors="ignore")

# =========================
# Force EXACT feature match with trained OLS model
# =========================
expected_features = ols_model.model.exog_names   # includes 'const'

# Align to model features (excluding constant first)
X_activation = X_activation.reindex(
    columns=[f for f in expected_features if f != "const"],
    fill_value=0
)

# Add constant
X_activation = sm.add_constant(X_activation, has_constant="add")

# Final strict alignment
X_activation = X_activation[expected_features]

# Ensure numeric
X_activation = X_activation.astype(float)

print("Activation shape:", X_activation.shape)
print("Model expects:", len(ols_model.params), "features")


Activation shape: (1, 16)
Model expects: 16 features


In [36]:
# =========================
# Prediction (model scale)
# =========================

# Predict normalized price
y_pred_norm = ols_model.predict(X_activation)

# Convert back to real Close Price
y_pred_real = price_scaler.inverse_transform(
    y_pred_norm.values.reshape(-1, 1)
).flatten()

activation_df["Predicted_Close_Price"] = y_pred_real


activation_df[["Date", "Close_Price", "Predicted_Close_Price"]].head()


,Date,Close_Price,Predicted_Close_Price
0,1970-02-17,11.62,10.734222


Create this table once (code or manual):

Model	RMSE	R²
ANN	lower	higher
OLS	higher	lower